## BDS HW 3

In [ ]:
!pip install requests
!pip install bs4
!pip install csv
!pip install PyPDF2
!pip install tika
!pip install tqdm
!pip install pandas
!pip install numpy

In [90]:
# imports
import requests 
from bs4 import BeautifulSoup
import csv
import os
import PyPDF2
from tika import parser
from tqdm import tqdm
import numpy as np
from math import log, e
import pandas as pd

In [ ]:
%config Completer.use_jedi = False

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
# URL from which pdfs to be downloaded
url = "http://proceedings.mlr.press/v70/"
  
# Requests URL and get response object
response = requests.get(url)
  
# Parse text obtained
soup = BeautifulSoup(response.text, 'html.parser')
  
# Find all hyperlinks present on webpage
links = soup.find_all('a')
  
i = 0
  
# From all links check for pdf link and
# if present download file
# check if links already exists
directory = os.getcwd()
for link in tqdm(links):
    if ('.pdf' in link.get('href', [])):
        i += 1
  
        # Get response object for link
        response = requests.get(link.get('href'))
  
        # Write content in pdf file
        # check if pdf exists already
        if not os.path.exists(directory + "/pdf"+str(i)+".pdf"):
            pdf = open("pdf"+str(i)+".pdf", 'wb')
            pdf.write(response.content)
            pdf.close()
  
print("All PDF files downloaded")

In [ ]:
directory = os.getcwd()
allwords = []
for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".pdf"):
        rawText = parser.from_file(filename)
        try:
            pdf_words = rawText['content'].split()
            allwords.extend([x.strip().lower() for x in pdf_words if x.strip().isalpha()])
        except AttributeError:
            print(f'There was an error with file: {filename}')

In [42]:
# clean words in allwords
with open('words_dictionary.json') as json_file:
    eng_dict = json.load(json_file)
    allwords_clean = []
    for word in allwords:
        if word.lower() in eng_dict:
            allwords_clean.append(word)

# save all words clean
text_file = open('all _words_clean.txt', 'w')
for elem in allwords_clean:
    text_file.write(elem+'\n')
text_file.close()

In [ ]:
print(f'The reduced word list after cleaning is: {len(allwords_clean)}\nFrom the original: {len(allwords)}')

1. What are the top 10 common words in the ICML papers?

In [43]:
# find the top 10 words
from collections import Counter

def getTop10(lst):
    counted = Counter(lst)
    return counted.most_common(10)

most_common = getTop10(allwords_clean)

In [46]:
print(f'The most common words used are:\n {most_common}')

The most common words used are:
 [('the', 188103), ('of', 103438), ('and', 88448), ('to', 62720), ('a', 59623), ('is', 56159), ('in', 52077), ('for', 45023), ('that', 36383), ('we', 35254)]


2. Let Z be a randomly selected word in a randomly selected ICML paper. Estimate the entropy of Z.

In [81]:
def entropy(items, base=None):
    n_items = len(items)
    
    if n_items <= 1:
        return 0
    
    value, counts = np.unique(items, return_counts=True)
    probs = counts / n_items
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0.

    # Compute entropy
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)

    return ent

In [84]:
print(entropy(allwords_clean))

6.725500029418058


3. Synthesize a random paragraph using the marginal distribution over words.

In [87]:
# create a DF with marginal distrabution probabilities and words
counted_words = Counter(allwords_clean)
marginal_dist_words = {}
n_words = len(allwords_clean)
for key, value in counted_words.items():
    marginal_dist_words[key] = value/n_words

In [91]:
marginal_dist_words_df = pd.DataFrame(marginal_dist_words.items(), columns=['word', 'distrabution'])

In [97]:
sampled_words = (marginal_dist_words_df.sample(n=10, weights='distrabution')['word']).to_list()

In [99]:
sentence = ''
for word in sampled_words:
    sentence += word+' '
sentence

'independent cause by depending Bandits we notice dimensional is on '

Problem 3: More on Kaggle Advanced Regression.